In [16]:
# -----------------------------------------------------
# 
# Generate lake model input files from CESM outputs
# 
# -----------------------------------------------------

import xarray as xr
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt


### 1. Read in data and extract the variables and coordinate(s) we want

In [17]:
# --- set the lake location (lat / lon, with lon -180 to 180)
latlake, lonlake = 36, -109
n_yrs_repeat = 10    # [yr] number of times to repeat the input data 
                     # (required because the model needs to be spun up)

# where to save the .txt file
# [ !! change to match your machine !! ]
savehere = "/Users/tylerkukla/Documents/GitHub/PRYSM/psm/lake_v2/clim_inputs"

# --- read in data
# [ !! change these to be specific to your machine !! ]
datpath = "/Users/tylerkukla/Documents/GitHub/coloplateau-isotopes/cesm_results"   # location of the cesm .nc file
casename_ctrl_suff = "modernTopo_280ppm"
casename_case_suff = "lowTopo_500ppm"
casename_case = f"CP_SLIM_{casename_case_suff}"                                         # name of the CESM simulation
casename_ctrl = f"CP_SLIM_{casename_ctrl_suff}"                                         # name of the CESM simulation
filename_case = f"{casename_case}.cam.h0.0020-0050._01-12_selectVars_climo.nc"
filename_ctrl = f"{casename_ctrl}.cam.h0.0020-0050._01-12_selectVars_climo.nc"

dsin_case = xr.open_dataset(os.path.join(datpath, filename_case))
dsin_ctrl = xr.open_dataset(os.path.join(datpath, filename_ctrl))

# convert longitude coords to -180 to 180
dsin_case["lon"] = ((dsin_case["lon"] + 180) % 360) - 180
dsin_ctrl["lon"] = ((dsin_ctrl["lon"] + 180) % 360) - 180

# sort the dataset by longitude
dsin_case = dsin_case.sortby("lon")
dsin_ctrl = dsin_ctrl.sortby("lon")
dsin_case


<xarray.Dataset>
Dimensions:    (lat: 96, lon: 144, lev: 30, ilev: 31, month: 12, nbnd: 2)
Coordinates:
  * lat        (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
  * lon        (lon) float64 -180.0 -177.5 -175.0 -172.5 ... 172.5 175.0 177.5
  * lev        (lev) float64 3.643 7.595 14.36 24.61 ... 936.2 957.5 976.3 992.6
  * ilev       (ilev) float64 2.255 5.032 10.16 18.56 ... 967.5 985.1 1e+03
  * month      (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: nbnd
Data variables: (12/62)
    gw         (month, lat) float64 ...
    hyam       (month, lev) float64 ...
    hybm       (month, lev) float64 ...
    P0         (month) float64 ...
    hyai       (month, ilev) float64 ...
    hybi       (month, ilev) float64 ...
    ...         ...
    SOLIN      (month, lat, lon) float32 ...
    T          (month, lev, lat, lon) float32 ...
    Z3         (month, lev, lat, lon) float32 ...
    TS         (month, lat, lon) float32 ...
    TREFHT     (month, lat, lon) float32 ...
    PRECT      (month, lat, lon) float32 ...

In [18]:
# --- extract the variables we need as inputs to the lake model
vars_to_keep = ["TREFHT",   # 2m air temperature (2m is the "reference height" or refht)
                "PRECT",    # precipitation
                "RELHUM",   # relative humidity 
                "U10",      # wind speed 
                "FSDS",     # surface incoming shortwave 
                "FLDS",     # surface incoming longwave
                "PS",       # surface pressure
                "QFLX",     # evap (for the runoff calculation)
                ]

dsin_case = dsin_case[vars_to_keep]
dsin_ctrl = dsin_ctrl[vars_to_keep]


In [19]:
# --- extract just the coordinate(s) that we want to analyze
dsin_case = dsin_case.sel(lat=latlake, lon=lonlake, method='nearest')
dsin_ctrl = dsin_ctrl.sel(lat=latlake, lon=lonlake, method='nearest')

# create a new ds to hold the converted values
ds_case = dsin_case.copy()
ds_ctrl = dsin_ctrl.copy()


# loop through all vars
Select one var to change, hold all others constant, generate the clim file

### Create a dataframe of the same format as the PRYSM example
For a table of inputs, see: https://agupubs.onlinelibrary.wiley.com/action/downloadSupplement?doi=10.1029%2F2018PA003413&file=palo20664-sup-0001-Supplementary.pdf

Dee et al. 2018 supplement states: For the environment sub-model, input rows A through I are required, and presently included. Rows I, J are required to calculate the water balance, and rows K through N are necessary to model stable water isotopes. If water balance and/or isotopes will not be modeled, these rows can either be left blank in the input file or filled with some sort of missing value.

#### Required rows: A-I
Year, Day of year, 2m Temperature, relative humidity, wind speed, surface shortwave, surface longwave, surface pressure, precipitation

#### optional for water balance
precipitation, basin wide runoff

#### optional for water isotopes
d18_p, dD_p, d18_runoff, dD_runoff


In [20]:
# [ convert units to play nice with the model ]

# loop through all vars, creating a dataset with entirely control data except for the single 
# var set to the case

for thisvar in vars_to_keep:

    ds = ds_ctrl.copy()
    ds[thisvar] = ds_case[thisvar].copy()

    # --- RELHUM: [%]
    # pull out just the surface data
    ds['RELHUM'] = ds['RELHUM'].sel(lev=1e3, method='nearest').copy()

    # values should already be in percent, but we need to make sure they
    # are within ~1 and 100 (trying to avoid values of 0 for the sake of 
    # potential divide by zero errors)
    ds['RELHUM'] = ds['RELHUM'].where(ds['RELHUM'] >= 1, 1).where(ds['RELHUM'] <= 100, 100).copy()

    # --- TREFHT: [degC]
    # convert from K to C
    # [ ! Dee et al. 2018 supplement notes temperature
    #     should be in deg C, but the example input is 
    #     in K, so we keep it K for now ! ]
    ds['TREFHT'] = ds['TREFHT'].copy() # - 273.15

    # --- PRECT: [mm]
    # convert m/s to mm (assume all months are 30 days for simplicity)
    # (conversion factors)
    s_per_day = 86400
    day_per_month = 30
    mm_per_m = 1e3
    # convert
    ds['PRECT'] = ds['PRECT'].copy() * mm_per_m * (s_per_day * day_per_month)

    # --- U10: [m/s]
    # U10 var is already in m/s
    ds['U10'] = ds['U10'].copy()

    # --- FLDS: [w/m2]
    # FLDS var is already in w/m2
    ds['FLDS'] = ds['FLDS'].copy()

    # --- FSDS: [w/m2]
    # FSDS var is already in w/m2
    ds['FSDS'] = ds['FSDS'].copy()

    # --- PS: [mb]
    # convert Pa to mb
    mb_per_Pa = 0.01

    # [ ! Dee et al. supplement notes surface 
    #     pressure should be in mb but the example
    #     input file is in Pa so we'll leave it in 
    #     Pa for now ! ]
    ds['PS'] = ds['PS'].copy() # * mb_per_Pa

    # --- Runoff: [mm/area of basin]
    # convert QFLX kg/m2/s to m/s
    density_h2o = 1000     # [kg / m3]

    # divide by water density to get m/s
    # then convert to mm
    ds['QFLX'] = (ds['QFLX'].copy() / density_h2o) * mm_per_m * (s_per_day * day_per_month)

    # get runoff
    ds['runoff'] = ds['PRECT'] - ds['QFLX']

    # force negative or zero values to something negligible
    ds['runoff'] = ds['runoff'].where(ds['runoff'] >= 0, 1e-3).copy()

    # --- [ create data table ]

    # [ !! NOTE CHECK THE COLUMN POSITION FOR RUNOFF VS PRECIPITATION !! ]

    # get rid of the level coord if it exists
    if 'lev' in ds.dims:
        ds = ds.drop_dims('lev').copy()
    # convert to dataframe
    df = ds.to_dataframe().reset_index()

    # convert month to day of year
    # assume all months have 30 days and we use 
    # the 15th day of the month for day of year
    df['day_of_year'] = df['month'] * 30 - 15

    # add a column for the year
    df['year'] = 1

    # add water iso columns and leave them blank
    df['d18_p'] = -10.
    df['dD_p'] = -20.
    df['d18_runoff'] = -11.
    df['dD_runoff'] = -22.

    # order columns
    column_order = ['year', 'day_of_year', 'TREFHT', 'RELHUM',
                    'U10', 'FSDS', 'FLDS', 'PS', # 'PRECT', 
                    'runoff', 'dD_p', 'd18_p', 'PRECT', 
                    'dD_runoff', 'd18_runoff']
    df = df[column_order]

    # round to two decimal places for consistency with Sylvia's inputs
    df = df.astype('float').round(2).copy()

    # repeat the data n_yrs_repeat times
    # (note, using 360 here is chosen to calibrate time steps with the 
    #  example in Sylvia's repo.)
    df = pd.concat([df.assign(year=df['year'] + i, day_of_year=360 * (i+1) - 360 + df['day_of_year']) for i in range(n_yrs_repeat)], ignore_index=True)


    # --- save the result
    # once with column headers for reference, 
    # once without column headers for the 
    # lake EBM input file

    # make filenames
    fn = os.path.join(savehere, f'sensitivityRun_{casename_ctrl_suff}+{thisvar}_{casename_case_suff}_input.txt')
    fn_header = os.path.join(savehere, f'sensitivityRun_{casename_ctrl_suff}+{thisvar}_{casename_case_suff}_input-withHeader.txt')
    # save the header-less version
    df.to_csv(fn, header=False, index=False, sep=' ')
    # save the header version
    df.to_csv(fn_header, header=True, index=False, sep=' ')


In [146]:
# -------------------------------------------